## Multi-head attention transformer
### Encoder and Decoder
### (With masking)

Pytorch's implementation (in built)

NOTE :- A new exmple must be used for testing the masked attention

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()

class TransformerModel(nn.Module):
    
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=4, nhead=2, num_encoder_layers=1, num_decoder_layers=1):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=0
        )
        self.fc = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt):

        print("Source embeddings :- \n")
        print(self.src_embedding(src))
        print()

        print("Target embeddings :- \n")
        print(self.tgt_embedding(tgt))
        print()


        src = self.src_embedding(src) + self.positional_encoding(src)
        tgt = self.tgt_embedding(tgt) + self.positional_encoding(tgt)

        print("Positional encoded source embeddings:")
        print(src)
        print()

        print("Positional encoded target embeddings:")
        print(tgt)
        print()

        output = self.transformer(src, tgt)
        output = self.fc(output)
        return output

    

src_vocab_size = 10  # Source language vocabulary size
tgt_vocab_size = 10  # Target language vocabulary size
d_model = 4  # Dimension of the model
num_heads = 2
model = TransformerModel(src_vocab_size, tgt_vocab_size, d_model=d_model, nhead=num_heads)
# Source sentence in the source language
src_sentence = torch.tensor([[0, 1, 2, 3]])  # Source sequence

# Target sentence in the target language (translation of the source sentence)
tgt_sentence = torch.tensor([[1, 0, 3, 3]])  # Target sequence

# Forward pass
output = model(src_sentence, tgt_sentence)
print(output)



Source embeddings :- 

tensor([[[-0.1965,  0.4001, -1.4294,  1.2002],
         [ 0.2407,  1.6451,  0.5767,  0.7134],
         [ 0.3766, -0.3972, -1.4901,  0.3670],
         [-0.1228,  0.0718, -0.7737, -0.0868]]], grad_fn=<EmbeddingBackward0>)

Target embeddings :- 

tensor([[[-0.2492, -1.9798, -0.3900, -0.5383],
         [ 1.2019,  2.0036, -0.5215,  1.9132],
         [-0.7919, -1.0487,  0.2215, -0.2312],
         [-0.7919, -1.0487,  0.2215, -0.2312]]], grad_fn=<EmbeddingBackward0>)

Positional encoded source embeddings:
tensor([[[-0.1965,  2.4001,  0.5706,  5.2002],
         [ 1.0822,  3.1854,  2.5867,  4.7134],
         [ 1.2859,  0.1867,  0.5299,  4.3668],
         [ 0.0183,  0.0818,  1.2563,  3.9128]]], grad_fn=<AddBackward0>)

Positional encoded target embeddings:
tensor([[[ 0.7508, -0.9798,  2.6100,  3.4617],
         [ 3.0434,  2.5439,  2.4885,  5.9131],
         [ 1.1174, -1.4648,  3.2415,  3.7686],
         [ 0.3492, -2.0387,  3.2515,  3.7683]]], grad_fn=<AddBackward0>)

query 

/Users/sreevaatsav/.pyenv/versions/project_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## Functions to get the manual calulations for each component

In [26]:
from torch import functional as F

# Using the state dictionary to get the intermediate outputs
state_dict = model.state_dict()

need_weights = False

src_mask = None

tgt_mask = None

memory_mask = None

embed_dim = 4

num_heads = 2


### Embeddings and Positional encoding

In [27]:
def get_embedding_outputs(src_sentence,  tgt_sentence, state_dict, d_model):

    src_vocab_embeds = state_dict["src_embedding.weight"]

    src_embedding = torch.zeros(src_sentence.size(0), src_sentence.size(1), d_model)

    print("Source sentence embedding")
    for i in range(src_sentence.size(0)):
        for j in range(src_sentence.size(1)):
            # Get the index for the current word in the sequence
            word_index = src_sentence[i, j].item()

            # Check if the index is within valid range
            if word_index < 0 or word_index >= src_vocab_embeds.size(0):
                raise ValueError(f"Invalid word index: {word_index}")

            # Lookup the corresponding embedding vector for the word
            src_embedding[i, j, :] = src_vocab_embeds[word_index, :]

            # Print intermediate results for debugging
            
            print(f"Word index: {word_index}, Embedding: {src_vocab_embeds[word_index, :]}")
    print()


    tgt_vocab_embeds = state_dict["tgt_embedding.weight"]

    tgt_embedding = torch.zeros(tgt_sentence.size(0), tgt_sentence.size(1), d_model)


    print("Target sentence embedding")
    for i in range(tgt_sentence.size(0)):
        for j in range(tgt_sentence.size(1)):
            # Get the index for the current word in the sequence
            word_index = tgt_sentence[i, j].item()

            # Check if the index is within valid range
            if word_index < 0 or word_index >= tgt_vocab_embeds.size(0):
                raise ValueError(f"Invalid word index: {word_index}")

            # Lookup the corresponding embedding vector for the word
            tgt_embedding[i, j, :] = tgt_vocab_embeds[word_index, :]

            # Print intermediate results for debugging
            print(f"Word index: {word_index}, Embedding: {tgt_vocab_embeds[word_index, :]}")
    print()


    pe = PositionalEncoding(d_model = d_model, dropout=0)

    pe_src_embeds = src_embedding + pe(src_sentence)

    pe_tgt_embeds = tgt_embedding + pe(tgt_sentence)

    print("PE source embeddings : \n")
    print(pe_src_embeds)
    print()

    print("PE target embeddings : \n")
    print(pe_tgt_embeds)
    print()

    return pe_src_embeds, pe_tgt_embeds



In [28]:
pe_src_embeds, pe_tgt_embeds = get_embedding_outputs(src_sentence,  tgt_sentence, state_dict, d_model = d_model)

Source sentence embedding
Word index: 0, Embedding: tensor([-0.1965,  0.4001, -1.4294,  1.2002])
Word index: 1, Embedding: tensor([0.2407, 1.6451, 0.5767, 0.7134])
Word index: 2, Embedding: tensor([ 0.3766, -0.3972, -1.4901,  0.3670])
Word index: 3, Embedding: tensor([-0.1228,  0.0718, -0.7737, -0.0868])

Target sentence embedding
Word index: 1, Embedding: tensor([-0.2492, -1.9798, -0.3900, -0.5383])
Word index: 0, Embedding: tensor([ 1.2019,  2.0036, -0.5215,  1.9132])
Word index: 3, Embedding: tensor([-0.7919, -1.0487,  0.2215, -0.2312])
Word index: 3, Embedding: tensor([-0.7919, -1.0487,  0.2215, -0.2312])

PE source embeddings : 

tensor([[[-0.1965,  2.4001,  0.5706,  5.2002],
         [ 1.0822,  3.1854,  2.5867,  4.7134],
         [ 1.2859,  0.1867,  0.5299,  4.3668],
         [ 0.0183,  0.0818,  1.2563,  3.9128]]])

PE target embeddings : 

tensor([[[ 0.7508, -0.9798,  2.6100,  3.4617],
         [ 3.0434,  2.5439,  2.4885,  5.9131],
         [ 1.1174, -1.4648,  3.2415,  3.7686],


## Encoder function to display the intermediate outputs and get the final outputs from the encoder

### Self attention 

In [29]:
def encoder_block_attn_output(x, state_dict, layer_num, embed_dim, num_heads, need_weights = False, src_mask = None):

    
    query_enc = key_enc = value_enc = x

    tgt_len, bsz, embed_dim = x.shape


    W_enc = state_dict["transformer.encoder.layers.{}.self_attn.in_proj_weight".format(layer_num)]
    b_enc = state_dict["transformer.encoder.layers.{}.self_attn.in_proj_bias".format(layer_num)]


    head_dim = embed_dim//num_heads

    tempop1 = query_enc[0]@W_enc.T
    tempop1 = tempop1.T

    Q_enc,K_enc,V_enc = tempop1.T.chunk(3, dim= -1)


    Q_enc = Q_enc.unsqueeze(0)
    K_enc = K_enc.unsqueeze(0)
    V_enc = V_enc.unsqueeze(0)


    Q_enc = Q_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_enc = K_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_enc = V_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_enc_{} = ".format(layer_num))
    print(Q_enc)
    print()

    print("K_enc_{} = ".format(layer_num))
    print(K_enc)
    print()

    print("V_enc_{} = ".format(layer_num))
    print(V_enc)
    print()

    src_len = K_enc.size(1)


    attn_mask = src_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)


    if need_weights is False:
        Q_enc1 = Q_enc.view(bsz, num_heads, tgt_len, head_dim)
        K_enc1 = K_enc.view(bsz, num_heads, src_len, head_dim)
        V_enc1 = V_enc.view(bsz, num_heads, src_len, head_dim)


        L, S = Q_enc1.size(-2), K_enc1.size(-2)

        scale_factor = 1 / math.sqrt(Q_enc1.size(-1)) 
        # scale_factor = 1
        attn_bias = torch.zeros(L, S, dtype=Q_enc1.dtype)

        if attn_mask is not None:
            if attn_mask.dtype == torch.bool:
                attn_mask.masked_fill_(attn_mask.logical_not(), float("-inf"))
            else:
                attn_bias += attn_mask

        attn_weight = Q_enc1 @ K_enc1.transpose(-2, -1) * scale_factor
        attn_weight += attn_bias
        attn_weight = torch.softmax(attn_weight, dim=-1)
        
        attn_output = attn_weight @ V_enc1

        attn_output = attn_output.permute(2, 0, 1, 3).contiguous().view(bsz * tgt_len, embed_dim)

        print("Encoder Attention output = ")
        print(attn_output)
        print()

        # print(src_len, pe_src_embeds.shape)
        return attn_output, src_len, head_dim, None
    
    else:

        B, Nt, E = Q_enc.shape

        Q_enc_scaled = Q_enc / math.sqrt(E)

        Q_enc_scaled

        if attn_mask is not None:
            temp_pdt_matrix_enc = torch.baddbmm(attn_mask, Q_enc_scaled, K_enc.transpose(-2, -1))
        else:
            temp_pdt_matrix_enc = torch.bmm(Q_enc_scaled, K_enc.transpose(-2, -1))


        # Obtaining the diagonal entries of the un-normalised product (K.Q_scaled)
        pdt_matrix_enc = torch.diagonal(temp_pdt_matrix_enc, dim1=-2, dim2=-1).unsqueeze(-1)

        attn_wt_matrix_enc = torch.nn.functional.softmax(pdt_matrix_enc, dim=-1)

        attn_enc_output = torch.bmm(attn_wt_matrix_enc, V_enc)

        attn_enc_output = attn_enc_output.transpose(0, 1).contiguous().view(tgt_len * bsz, embed_dim)

        print("Encoder Attention output = ")
        print(attn_enc_output)
        print()

        return attn_enc_output, src_len, head_dim, attn_wt_matrix_enc

    

In [30]:
# need_weights = True

attn_enc_output, src_len, head_dim, attn_weights = encoder_block_attn_output(pe_src_embeds, state_dict, layer_num = 0, need_weights = need_weights, embed_dim=embed_dim, num_heads=num_heads)


Q_enc_0 = 
tensor([[[-1.8032, -0.8990]],

        [[ 2.2382,  4.2951]],

        [[-0.3593, -0.7384]],

        [[ 2.4799,  4.1834]],

        [[-1.3313, -2.1788]],

        [[ 1.5391,  2.7540]],

        [[-1.4253, -1.8787]],

        [[ 1.9033,  2.2597]]])

K_enc_0 = 
tensor([[[ 0.5545, -2.5529]],

        [[ 1.4227, -1.0407]],

        [[ 1.7060, -3.1221]],

        [[ 1.1894, -1.5360]],

        [[-0.3429, -1.8059]],

        [[ 0.9272, -1.7877]],

        [[-0.2644, -2.0345]],

        [[ 0.4344, -1.2102]]])

V_enc_0 = 
tensor([[[-1.7160, -1.7735]],

        [[ 2.4448, -2.5212]],

        [[ 0.2962, -1.9649]],

        [[ 2.1180, -2.5557]],

        [[-1.1142, -0.9874]],

        [[ 2.7670, -0.9002]],

        [[-1.3984, -0.4769]],

        [[ 2.3564, -1.3284]]])

Encoder Attention output = 
tensor([[-1.7160, -1.7735,  2.4448, -2.5212],
        [ 0.2962, -1.9649,  2.1180, -2.5557],
        [-1.1142, -0.9874,  2.7670, -0.9002],
        [-1.3984, -0.4769,  2.3564, -1.3284]])



### Post self attention in encoder

In [31]:
def encoder_block_post_attn_output(x, attn_enc_output, state_dict, layer_num, bsz, tgt_len):

    op_enc_1 = torch.matmul(attn_enc_output, state_dict["transformer.encoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]

    attn_enc_output = op_enc_1.view(tgt_len, bsz, attn_enc_output.size(1))


    # Here src is the original passed inputs to the 1st transformer encoder layer which 
    # are pe_src_embeds 
    output_enc_1 = attn_enc_output + x
    output_enc_1

    linear_result_enc_1 = output_enc_1*state_dict["transformer.encoder.layers.{}.norm1.weight".format(layer_num)] + state_dict["transformer.encoder.layers.{}.norm1.bias".format(layer_num)]

    # Layer normalization from Torch's implementation
    layernorm_enc_1 = torch.nn.LayerNorm(normalized_shape=linear_result_enc_1.shape[2:])
    linear_op_enc_1 = layernorm_enc_1(linear_result_enc_1)


    # Manual Layer Normalization
    x = linear_result_enc_1

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_enc_1.weight
    b = layernorm_enc_1.bias

    linear_result_enc_1f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_enc_1f.mean(dim=-1, keepdim=True)
    std = linear_result_enc_1f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_enc_1 = (linear_result_enc_1f - mean) / (std + epsilon) * w + b


    op_enc_1 = torch.matmul(normalized_result_enc_1, state_dict["transformer.encoder.layers.{}.linear1.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.linear1.bias".format(layer_num)]
    op_enc_1_relu = torch.nn.functional.relu(op_enc_1)
    op_enc_2 = torch.matmul(op_enc_1_relu, state_dict["transformer.encoder.layers.{}.linear2.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.linear2.bias".format(layer_num)]


    output_enc_2 = op_enc_2 + linear_op_enc_1
    output_enc_2_norm = output_enc_2*state_dict["transformer.encoder.layers.{}.norm2.weight".format(layer_num)] + state_dict["transformer.encoder.layers.{}.norm2.bias".format(layer_num)]

    # Layer normalization from Torch's implementation
    layernorm_enc_final = torch.nn.LayerNorm(normalized_shape=output_enc_2_norm.shape[2:])
    output_enc_final = layernorm_enc_final(output_enc_2_norm)


    # Manual Layer Normalization 
    x = output_enc_2_norm

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_enc_final.weight
    b = layernorm_enc_final.bias

    linear_result_enc_2 = w*x + b

    epsilon = 1e-5  
    mean = linear_result_enc_2.mean(dim=-1, keepdim=True)
    std = linear_result_enc_2.std(dim=-1, unbiased=False, keepdim=True)
    output_enc_final = (linear_result_enc_2 - mean) / (std + epsilon) * w + b

    print("Final Encoder {} Output :".format(layer_num))
    print("norm2(norm1(x + self_atten(x)) + feed_fwd_op)\n")
    print(output_enc_final)
    print()

    return output_enc_final


In [32]:
tgt_len, bsz, embed_dim = pe_src_embeds.shape
output_enc_final = encoder_block_post_attn_output(pe_src_embeds, attn_enc_output, state_dict, layer_num = 0 , bsz = bsz, tgt_len = tgt_len)

Final Encoder 0 Output :
norm2(norm1(x + self_atten(x)) + feed_fwd_op)

tensor([[[-1.1375,  0.0461, -0.4810,  1.5724],
         [-0.9991, -0.5350, -0.1102,  1.6442],
         [-0.3924, -0.7949, -0.5264,  1.7137],
         [-0.9869, -0.7650,  0.2028,  1.5491]]], grad_fn=<AddBackward0>)



## Decoder functions to display the intermediate outputs and get the final outputs from the decoder


### Self attention outputs from a decoder block

In [33]:
def decoder_block_self_attn_output(x, state_dict, layer_num, tgt_mask = None,need_weights = False):

    query_dec = key_dec = value_dec = x

    tgt_len, bsz, embed_dim = x.shape

    W_dec = state_dict["transformer.decoder.layers.{}.self_attn.in_proj_weight".format(layer_num)]
    b_dec = state_dict["transformer.decoder.layers.{}.self_attn.in_proj_bias".format(layer_num)]


    head_dim = embed_dim//num_heads

    tempop1 = query_dec[0]@W_dec.T
    tempop1 = tempop1.T

    Q_dec,K_dec,V_dec = tempop1.T.chunk(3, dim= -1)


    Q_dec = Q_dec.unsqueeze(0)
    K_dec = K_dec.unsqueeze(0)
    V_dec = V_dec.unsqueeze(0)


    Q_dec = Q_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_dec = K_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_dec = V_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_dec_{} = ".format(layer_num))
    print(Q_dec)
    print()

    print("K_dec_{} = ".format(layer_num))
    print(K_dec)
    print()

    print("V_dec_{} = ".format(layer_num))
    print(V_dec)
    print()

    src_len = K_dec.size(1)


    attn_mask = tgt_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)



    if need_weights is False:
        Q_dec1 = Q_dec.view(bsz, num_heads, tgt_len, head_dim)
        K_dec1 = K_dec.view(bsz, num_heads, src_len, head_dim)
        V_dec1 = V_dec.view(bsz, num_heads, src_len, head_dim)


        L, S = Q_dec1.size(-2), K_dec1.size(-2)

        scale_factor = 1 / math.sqrt(Q_dec1.size(-1)) 
        # scale_factor = 1
        attn_bias = torch.zeros(L, S, dtype=Q_dec1.dtype)

        if attn_mask is not None:
            if attn_mask.dtype == torch.bool:
                attn_mask.masked_fill_(attn_mask.logical_not(), float("-inf"))
            else:
                attn_bias += attn_mask


        attn_weight = Q_dec1 @ K_dec1.transpose(-2, -1) * scale_factor
        attn_weight += attn_bias
        attn_weight = torch.softmax(attn_weight, dim=-1)
        
        attn_output = attn_weight @ V_dec1

        attn_output = attn_output.permute(2, 0, 1, 3).contiguous().view(bsz * tgt_len, embed_dim)

        print("Decoder Self Attention = ")
        print(attn_output)
        print()

        op_dec_1 = torch.matmul(attn_output, state_dict["transformer.decoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]
        attn_dec_output = op_dec_1.view(tgt_len, bsz, attn_output.size(1))

        return attn_dec_output, None
    
    else:


        B, Nt, E = Q_dec.shape

        Q_dec_scaled = Q_dec / math.sqrt(E)


        if attn_mask is not None:
            temp_pdt_matrix_dec = torch.baddbmm(attn_mask, Q_dec_scaled, K_dec.transpose(-2, -1))
        else:
            temp_pdt_matrix_dec = torch.bmm(Q_dec_scaled, K_dec.transpose(-2, -1))
            

        # Obtaining the diagonal entries of the un-normalised product (K.Q_scaled)
        pdt_matrix_dec = torch.diagonal(temp_pdt_matrix_dec, dim1=-2, dim2=-1).unsqueeze(-1)

        print("Q_scaled @ Kt = ", pdt_matrix_dec)

        attn_wt_matrix_dec = torch.nn.functional.softmax(pdt_matrix_dec, dim=-1)

        attn_dec_output = torch.bmm(attn_wt_matrix_dec, V_dec)

        attn_dec_output = attn_dec_output.transpose(0, 1).contiguous().view(tgt_len * bsz, embed_dim)

        print("Decoder Attention output = ")
        print(attn_wt_matrix_dec)
        print()


        op_dec_1 = torch.matmul(attn_dec_output, state_dict["transformer.decoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]
        attn_dec_output = op_dec_1.view(tgt_len, bsz, attn_dec_output.size(1))
    

        return attn_dec_output, attn_wt_matrix_dec




# query_dec = key_dec = value_dec = pe_tgt_embeds 

# tgt_len, bsz, embed_dim = pe_src_embeds.shape
    

In [34]:
need_weights

False

In [35]:
# need_weights = True

self_attn_dec, dec_sa_wts = decoder_block_self_attn_output(pe_tgt_embeds, state_dict, layer_num = 0, need_weights = need_weights)

Q_dec_0 = 
tensor([[[-0.5715, -0.9803]],

        [[-0.2668, -2.1467]],

        [[ 0.6712,  1.4975]],

        [[-1.1324, -1.3539]],

        [[-1.1546, -1.6399]],

        [[-0.3026, -2.6020]],

        [[-0.9442, -1.7529]],

        [[-0.1179, -2.8550]]])

K_dec_0 = 
tensor([[[ 0.5695, -0.4099]],

        [[ 1.4165,  1.8115]],

        [[-0.6667,  2.4925]],

        [[ 2.2106,  0.3611]],

        [[ 0.8026, -0.7249]],

        [[ 1.6418,  2.0580]],

        [[ 0.9950, -1.1761]],

        [[ 1.5863,  2.6246]]])

V_dec_0 = 
tensor([[[-0.6472,  1.7969]],

        [[-1.4070, -0.1967]],

        [[ 0.9499, -1.1671]],

        [[-4.2108, -1.5491]],

        [[-1.0828,  2.1850]],

        [[-1.5985, -0.3161]],

        [[-1.1503,  2.9487]],

        [[-1.0779,  0.0906]]])

Decoder Self Attention = 
tensor([[-0.6472,  1.7969, -1.4070, -0.1967],
        [ 0.9499, -1.1671, -4.2108, -1.5491],
        [-1.0828,  2.1850, -1.5985, -0.3161],
        [-1.1503,  2.9487, -1.0779,  0.0906]])



In [36]:
self_attn_dec

tensor([[[-0.7972, -0.8369, -2.9348,  1.6432],
         [ 0.5649,  3.8928, -2.7056, -2.2094],
         [-0.8494, -1.3390, -3.7335,  2.1888],
         [-1.3322, -2.1464, -3.6756,  2.9461]]])

In [37]:
def dec_post_self_attn(self_attn_dec, x, state_dict, layer_num):
    output_dec_1 = self_attn_dec + x

    linear_result_dec_1 = output_dec_1*state_dict["transformer.decoder.layers.{}.norm1.weight".format(layer_num)] + state_dict["transformer.decoder.layers.{}.norm1.bias".format(layer_num)]

    layernorm_dec_1 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_1.shape[2:])
    linear_op_dec_1 = layernorm_dec_1(linear_result_dec_1)

    # From torch's implementation
    # print(linear_op_dec_1)

    x = linear_result_dec_1

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_dec_1.weight
    b = layernorm_dec_1.bias

    linear_result_dec_1f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_1f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_1f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_1 = (linear_result_dec_1f - mean) / (std + epsilon) * w + b


    print("Decoder_{} norm1(x + sa(x))".format(layer_num))
    print(normalized_result_dec_1)
    print()

    return normalized_result_dec_1

In [38]:
x_dec = dec_post_self_attn(self_attn_dec, pe_tgt_embeds, state_dict, layer_num = 0)

Decoder_0 norm1(x + sa(x))
tensor([[[-0.2967, -0.9738, -0.4032,  1.6737],
         [ 0.0951,  1.2893, -1.5199,  0.1355],
         [-0.1441, -1.0976, -0.3800,  1.6218],
         [-0.3173, -1.1214, -0.1770,  1.6157]]], grad_fn=<AddBackward0>)



## Cross attention in decoder 
### query, key, value =  x, mem, mem 

### Cross attention between the encoder's final output and the decoder layer 

In [39]:

memory = output_enc_final
x_dec

tensor([[[-0.2967, -0.9738, -0.4032,  1.6737],
         [ 0.0951,  1.2893, -1.5199,  0.1355],
         [-0.1441, -1.0976, -0.3800,  1.6218],
         [-0.3173, -1.1214, -0.1770,  1.6157]]], grad_fn=<AddBackward0>)

In [42]:
def decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num, tgt_len, src_len, head_dim, memory_mask = None,need_weights = False):

    query_dec_mha = x_dec
    key_dec_mha, value_dec_mha = memory, memory


    W_dec_mha = state_dict["transformer.decoder.layers.{}.multihead_attn.in_proj_weight".format(layer_num)]
    b_dec_mha = state_dict["transformer.decoder.layers.{}.multihead_attn.in_proj_bias".format(layer_num)]


    ########## CHANGE FROM SELF ATTENTION ############

    W_q, W_k, W_v = W_dec_mha.chunk(3)

    Q_dec_mha = query_dec_mha[0]@W_q.T

    K_dec_mha = key_dec_mha[0]@W_k.T

    V_dec_mha = value_dec_mha[0]@W_v.T

    #################################################

    K_dec_mha = K_dec_mha.unsqueeze(0)
    V_dec_mha = V_dec_mha.unsqueeze(0)
    Q_dec_mha = Q_dec_mha.unsqueeze(0)

    Q_dec_mha = Q_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_dec_mha = K_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_dec_mha = V_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_dec_{} = ".format(layer_num))
    print(Q_dec_mha)
    print()

    print("K_dec_{} = ".format(layer_num))
    print(K_dec_mha)
    print()

    print("V_dec_{} = ".format(layer_num))
    print(V_dec_mha)
    print()


    attn_mask = memory_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)


    if need_weights is False:
        Q_dec_mha1 = Q_dec_mha.view(bsz, num_heads, tgt_len, head_dim)
        K_dec_mha1 = K_dec_mha.view(bsz, num_heads, src_len, head_dim)
        V_dec_mha1 = V_dec_mha.view(bsz, num_heads, src_len, head_dim)

        L, S = Q_dec_mha1.size(-2), K_dec_mha1.size(-2)

        scale_factor = 1 / math.sqrt(Q_dec_mha1.size(-1)) 
        # scale_factor = 1

        attn_bias = torch.zeros(L, S, dtype=Q_dec_mha1.dtype)
        if attn_mask is not None:
            if attn_mask.dtype == torch.bool:
                attn_mask.masked_fill_(attn_mask.logical_not(), float("-inf"))
            else:
                attn_bias += attn_mask


        attn_weight = Q_dec_mha1 @ K_dec_mha1.transpose(-2, -1) * scale_factor
        attn_weight += attn_bias
        attn_weight = torch.softmax(attn_weight, dim=-1)
        attn_output_dec_mha = attn_weight @ V_dec_mha1

        attn_output_dec_mha = attn_output_dec_mha.permute(2, 0, 1, 3).view(bsz * tgt_len, embed_dim)
        
        print("Cross attention in decoder_{}".format(layer_num))
        print(attn_output_dec_mha)
        print()

        op_dec_mha_1 = torch.matmul(attn_output_dec_mha, state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.bias".format(layer_num)]

        attn_dec_mha_output = op_dec_mha_1.view(tgt_len, bsz, attn_output_dec_mha.size(1))

        return attn_dec_mha_output, None
    
    else: 

        B, Nt, E = Q_dec_mha.shape

        Q_dec_mha_scaled = Q_dec_mha / math.sqrt(E)


        if attn_mask is not None:
            temp_pdt_matrix_dec_mha = torch.baddbmm(attn_mask, Q_dec_mha_scaled, K_dec_mha.transpose(-2, -1))
        else:
            temp_pdt_matrix_dec_mha = torch.bmm(Q_dec_mha_scaled, K_dec_mha.transpose(-2, -1))
            

        # Obtaining the diagonal entries of the un-normalised product (K.Q_scaled)
        pdt_matrix_dec_mha = torch.diagonal(temp_pdt_matrix_dec_mha, dim1=-2, dim2=-1).unsqueeze(-1)

        print("Q_scaled @ Kt = ", pdt_matrix_dec_mha)

        attn_wt_matrix_dec_mha = torch.nn.functional.softmax(pdt_matrix_dec_mha, dim=-1)

        attn_dec_mha_output = torch.bmm(attn_wt_matrix_dec_mha, V_dec_mha)

        attn_dec_mha_output = attn_dec_mha_output.transpose(0, 1).contiguous().view(tgt_len * bsz, embed_dim)

        print("Decoder mha Attention output = ")
        print(attn_dec_mha_output)
        print()


        op_dec_mha = torch.matmul(attn_dec_mha_output, state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.bias".format(layer_num)]
        attn_dec_output = op_dec_mha.view(tgt_len, bsz, attn_dec_mha_output.size(1))


        return attn_dec_output , attn_wt_matrix_dec_mha

        





In [43]:
# need_weights = True

attn_dec_mha_output, attn_dec_mha_wts = decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num = 0, tgt_len = tgt_len, src_len = src_len, head_dim = head_dim, need_weights = need_weights)

Q_dec_0 = 
tensor([[[ 1.1551, -0.5112]],

        [[ 0.2386, -0.9911]],

        [[ 0.2198, -0.2264]],

        [[-0.3958,  0.5742]],

        [[ 1.1903, -0.4910]],

        [[ 0.3076, -1.0038]],

        [[ 1.0901, -0.4674]],

        [[ 0.2824, -1.0465]]], grad_fn=<TransposeBackward0>)

K_dec_0 = 
tensor([[[-0.0704,  0.2770]],

        [[ 1.2327, -0.4538]],

        [[-0.1660,  0.3082]],

        [[ 0.8527, -0.2363]],

        [[-0.4686, -0.0289]],

        [[ 0.3370, -0.2864]],

        [[-0.1386,  0.3806]],

        [[ 0.6938, -0.0846]]], grad_fn=<TransposeBackward0>)

V_dec_0 = 
tensor([[[ 0.1339,  0.0898]],

        [[ 0.1141,  0.5882]],

        [[ 0.0868,  0.4066]],

        [[-0.0548,  0.3838]],

        [[ 0.3645,  0.3931]],

        [[-0.1983, -0.0611]],

        [[-0.0098,  0.5592]],

        [[-0.1135,  0.3421]]], grad_fn=<TransposeBackward0>)

Cross attention in decoder_0
tensor([[ 0.1339,  0.0898,  0.1141,  0.5882],
        [ 0.0868,  0.4066, -0.0548,  0.3838],
        [

In [44]:
def decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num):

    output_dec_2 = attn_dec_mha_output + x_dec

    linear_result_dec_2 = output_dec_2*state_dict["transformer.decoder.layers.{}.norm2.weight".format(layer_num)] + state_dict["transformer.decoder.layers.{}.norm2.bias".format(layer_num)]

    # Layer normalization from Torch's implementation 
    layernorm_dec_2 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_2.shape[2:])
    linear_op_dec_2 = layernorm_dec_2(linear_result_dec_2)

    x = linear_result_dec_2
    w = layernorm_dec_2.weight
    b = layernorm_dec_2.bias

    linear_result_dec_2f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_2f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_2f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_2 = (linear_result_dec_2f - mean) / (std + epsilon) * w + b

    print("norm2(x' + mha(x', mem)), \n where x' = Decoder_curr_layer norm1(x + sa(x))")
    print(normalized_result_dec_2)
    print("\n\n")

    x_dec2_norm = normalized_result_dec_2

    op_dec_1 = torch.matmul(x_dec2_norm, state_dict["transformer.decoder.layers.{}.linear1.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.linear1.bias".format(layer_num)]
    op_dec_1_relu = torch.nn.functional.relu(op_dec_1)
    op_dec_2 = torch.matmul(op_dec_1_relu, state_dict["transformer.decoder.layers.{}.linear2.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.linear2.bias".format(layer_num)]

    ff_dec = op_dec_2

    x_dec3_unorm = x_dec2_norm + ff_dec

    linear_result_dec_3 = x_dec3_unorm*state_dict["transformer.decoder.layers.0.norm3.weight"] + state_dict["transformer.decoder.layers.0.norm3.bias"]

    # Layer normalization from Torch's implementation 
    layernorm_dec_3 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_3.shape[2:])
    linear_op_dec_3 = layernorm_dec_3(linear_result_dec_3)

    # From torch's implementation
    x = linear_result_dec_3

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_dec_3.weight
    b = layernorm_dec_3.bias

    linear_result_dec_3f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_3f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_3f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_3 = (linear_result_dec_3f - mean) / (std + epsilon) * w + b

    print("norm3(x'' + ff(x'')) \n where, x'' = Decoder_curr_layer norm2(x' + mha(x'))")
    print(normalized_result_dec_3)
    print()

    return normalized_result_dec_3

def feef_fwd_transformer(dec_output_final, state_dict):

    final_op = dec_output_final@state_dict["fc.weight"].T + state_dict["fc.bias"]

    return final_op

In [45]:
final_attn_op_decoder = decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num = 0)

norm2(x' + mha(x', mem)), 
 where x' = Decoder_curr_layer norm1(x + sa(x))
tensor([[[-0.0528, -1.2301, -0.2699,  1.5528],
         [ 0.2214,  1.0573, -1.6432,  0.3646],
         [-0.2337, -0.7955, -0.6650,  1.6942],
         [-0.1521, -1.0451, -0.4425,  1.6397]]], grad_fn=<AddBackward0>)



norm3(x'' + ff(x'')) 
 where, x'' = Decoder_curr_layer norm2(x' + mha(x'))
tensor([[[ 0.0823, -1.1172, -0.5336,  1.5685],
         [ 0.3409,  0.7710, -1.7116,  0.5997],
         [-0.0431, -0.7461, -0.8564,  1.6456],
         [ 0.0110, -0.9569, -0.6753,  1.6212]]], grad_fn=<AddBackward0>)



In [46]:
final_op = feef_fwd_transformer(final_attn_op_decoder, state_dict)

In [47]:
final_op

tensor([[[-0.8690,  0.2952,  0.6433,  0.9624, -0.4184,  1.0310, -0.2743,
           0.2156,  0.6588, -0.5275],
         [-0.1098,  0.0060, -0.0221,  0.2819, -0.9641,  0.5592, -0.5691,
          -0.1397,  0.5262,  0.3902],
         [-0.7065,  0.2620,  0.5209,  0.8902, -0.5664,  1.0184, -0.2785,
           0.0787,  0.5858, -0.3458],
         [-0.7971,  0.2843,  0.5912,  0.9347, -0.4833,  1.0307, -0.2683,
           0.1500,  0.6215, -0.4467]]], grad_fn=<AddBackward0>)

########################################################################

### Examaple :- 

Transformer with 3 encoders and 3 decoders

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()

class TransformerModel(nn.Module):
    
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=4, nhead=2, num_encoder_layers=1, num_decoder_layers=1):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=0
        )
        self.fc = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt):

        print("Source embeddings :- \n")
        print(self.src_embedding(src))
        print()

        print("Target embeddings :- \n")
        print(self.tgt_embedding(tgt))
        print()


        src = self.src_embedding(src) + self.positional_encoding(src)
        tgt = self.tgt_embedding(tgt) + self.positional_encoding(tgt)

        print("Positional encoded source embeddings:")
        print(src)
        print()

        print("Positional encoded target embeddings:")
        print(tgt)
        print()

        output = self.transformer(src, tgt)
        output = self.fc(output)
        return output

    

src_vocab_size = 10  # Source language vocabulary size
tgt_vocab_size = 10  # Target language vocabulary size
d_model = 4  # Dimension of the model
num_heads = 2
num_encoder_layers = 3
num_decoder_layers = 3

need_weights = False

src_mask = None
tgt_mask = None
memory_mask = None


model = TransformerModel(src_vocab_size, tgt_vocab_size, d_model=d_model, nhead=num_heads, num_encoder_layers = num_encoder_layers, num_decoder_layers=num_decoder_layers)
# Source sentence in the source language
src_sentence = torch.tensor([[0, 1, 2, 3]])  # Source sequence

# Target sentence in the target language (translation of the source sentence)
tgt_sentence = torch.tensor([[1, 0, 3, 3]])  # Target sequence

# Forward pass
output = model(src_sentence, tgt_sentence)
print(output)


Source embeddings :- 

tensor([[[-0.2205,  1.7253, -0.2514,  0.9972],
         [ 0.1335, -0.5650, -0.7537,  0.0105],
         [-0.8226,  0.9250,  1.7916, -0.4149],
         [ 0.2400, -2.0239, -0.6187, -0.7110]]], grad_fn=<EmbeddingBackward0>)

Target embeddings :- 

tensor([[[ 0.0958, -0.3552,  1.2360, -0.8826],
         [ 0.0204,  1.1688, -0.0480, -1.3756],
         [-0.1728,  0.3034, -1.1816, -1.1548],
         [-0.1728,  0.3034, -1.1816, -1.1548]]], grad_fn=<EmbeddingBackward0>)

Positional encoded source embeddings:
tensor([[[-0.2205,  3.7253,  1.7486,  4.9972],
         [ 0.9750,  0.9753,  1.2563,  4.0105],
         [ 0.0867,  1.5088,  3.8116,  3.5849],
         [ 0.3811, -2.0139,  1.4113,  3.2885]]], grad_fn=<AddBackward0>)

Positional encoded target embeddings:
tensor([[[ 1.0958,  0.6448,  4.2360,  3.1174],
         [ 1.8619,  1.7091,  2.9620,  2.6244],
         [ 1.7365, -0.1128,  1.8384,  2.8450],
         [ 0.9683, -0.6866,  1.8484,  2.8448]]], grad_fn=<AddBackward0>)

query 

/Users/sreevaatsav/.pyenv/versions/project_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [53]:
def get_all_intermediate_outputs(src_sentence, tgt_sentence,d_model, model, num_encoder_layers , num_decoder_layers):

    state_dict = model.state_dict()

    pe_src_embeds, pe_tgt_embeds = get_embedding_outputs(src_sentence,  tgt_sentence, state_dict, d_model = d_model)

    print("###"*25)
    print("### Encoder Start ###")
    print()

    x_enc = pe_src_embeds

    for lno in range(num_encoder_layers):
        attn_enc_output, src_len, head_dim, attn_weights = encoder_block_attn_output(x_enc, state_dict, layer_num = lno, need_weights = False, embed_dim = d_model, num_heads= num_heads, src_mask=None)

        if lno == 0:
            tgt_len, bsz, embed_dim = x_enc.shape

        output_enc_final = encoder_block_post_attn_output(x_enc, attn_enc_output, state_dict, layer_num = lno , bsz = bsz, tgt_len = tgt_len)

        x_enc = output_enc_final

    
    print("### Encoder Done ###")
    print("\n\n\n")
    print("### Decoder Start ###")

    x_dec = pe_tgt_embeds
    memory = x_enc

    for lno in range(num_decoder_layers):

        self_attn_dec, dec_sa_wts = decoder_block_self_attn_output(x_dec, state_dict, layer_num = lno, need_weights = False, tgt_mask=None)
        x_dec = dec_post_self_attn(self_attn_dec, x_dec, state_dict, layer_num = lno)

        attn_dec_mha_output, attn_dec_mha_wts = decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num = lno, tgt_len = tgt_len, src_len = src_len, head_dim = head_dim, need_weights = False, memory_mask=None)
        final_op = decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num = lno)

        print(pe_tgt_embeds.shape, final_op.shape)
        x_dec = final_op


    final_op = feef_fwd_transformer(final_op, state_dict)

    print("### Decoder Done ###")


    return final_op

    

In [54]:
d_model = 4
num_encoder_layers = 3
num_decoder_layers = 3

get_all_intermediate_outputs(src_sentence, tgt_sentence, model = model, num_encoder_layers = num_encoder_layers , num_decoder_layers = num_encoder_layers, d_model=d_model)

Source sentence embedding
Word index: 0, Embedding: tensor([-0.2205,  1.7253, -0.2514,  0.9972])
Word index: 1, Embedding: tensor([ 0.1335, -0.5650, -0.7537,  0.0105])
Word index: 2, Embedding: tensor([-0.8226,  0.9250,  1.7916, -0.4149])
Word index: 3, Embedding: tensor([ 0.2400, -2.0239, -0.6187, -0.7110])

Target sentence embedding
Word index: 1, Embedding: tensor([ 0.0958, -0.3552,  1.2360, -0.8826])
Word index: 0, Embedding: tensor([ 0.0204,  1.1688, -0.0480, -1.3756])
Word index: 3, Embedding: tensor([-0.1728,  0.3034, -1.1816, -1.1548])
Word index: 3, Embedding: tensor([-0.1728,  0.3034, -1.1816, -1.1548])

PE source embeddings : 

tensor([[[-0.2205,  3.7253,  1.7486,  4.9972],
         [ 0.9750,  0.9753,  1.2563,  4.0105],
         [ 0.0867,  1.5088,  3.8116,  3.5849],
         [ 0.3811, -2.0139,  1.4113,  3.2885]]])

PE target embeddings : 

tensor([[[ 1.0958,  0.6448,  4.2360,  3.1174],
         [ 1.8619,  1.7091,  2.9620,  2.6244],
         [ 1.7365, -0.1128,  1.8384,  2.845

tensor([[[-0.1736, -0.6635, -0.7838, -0.4535, -1.0496, -0.8957,  0.4070,
           0.4802,  0.3982, -0.3234],
         [-0.1878, -0.6338, -0.7410, -0.4279, -1.0766, -0.9809,  0.3347,
           0.5342,  0.4060, -0.4664],
         [-0.1235, -0.6708, -0.8959, -0.4473, -0.9361, -0.7919,  0.5501,
           0.3801,  0.3818, -0.0212],
         [-0.1777, -0.6718, -0.6719, -0.4529, -1.0911, -0.8548,  0.3259,
           0.5181,  0.4016, -0.4310]]], grad_fn=<AddBackward0>)